In [16]:
from torch import nn, optim

class SplitNN(nn.Module):
    def __init__(self):
        super(SplitNN, self).__init__()
        self.first_part = nn.Sequential(
            nn.Linear(784, 500),
            nn.ReLU(),
        )
        self.second_part = nn.Sequential(
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 10),
            nn.Softmax(dim=-1),
        )
        
    def forward(self, x):
        #x = torch.flatten(x, start_dim=1)  # Flatten input from 28x28 to 784
        return self.second_part(self.first_part(x))

target_model = SplitNN()

In [18]:
class Attacker(nn.Module):
    def __init__(self):
        super(Attacker, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(500, 1000),
            nn.ReLU(),
            nn.Linear(1000, 784),
        )

    def forward(self, x):
        return self.layers(x)

In [20]:
attacker = Attacker()
optimiser = optim.Adam(attacker.parameters(), lr=1e-4)

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download and load the EMNIST dataset
emnist_train = datasets.EMNIST(root='data', split='letters', train=True, download=True, transform=transform)
emnist_test = datasets.EMNIST(root='data', split='letters', train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(emnist_train, batch_size=64, shuffle=True)
test_loader = DataLoader(emnist_test, batch_size=64, shuffle=False)

100%|██████████| 561753746/561753746 [01:07<00:00, 8269395.22it/s] 


Extracting data\EMNIST\raw\gzip.zip to data\EMNIST\raw


In [24]:
for data, targets in train_loader:
    print(data.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 2

In [14]:
for data, targets in train_loader:
  # Reset gradients
  optimiser.zero_grad()
  # First, get outputs from the target model
  target_outputs = target_model.first_part(data)
  # Next, recreate the data with the attacker
  attack_outputs = attacker(target_outputs)
  # We want attack outputs to resemble the original data
  loss = ((data - attack_outputs)**2).mean()
  # Update the attack model
  loss.backward()
  optimiser.step()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1792x28 and 784x500)

In [ ]:
for data, targets in mnist_test_loader:
  target_outputs = target_model.first_part(data)
  recreated_data = attacker(target_outputs)